# Validate Monday.com tasks for integration issues

In [ ]:
%load_ext nb_black

import logging

import pandas as pd
import numpy as np

import prefect
from prefect import task, Flow, Parameter, unmapped
from prefect.executors import LocalExecutor, LocalDaskExecutor
from prefect.utilities.logging import get_logger

from datetime import timedelta, datetime
from box import Box

from mondaydotcom_utils.formatted_value import (
    FormattedValue,
    get_col_defs,
    get_items_by_board,
)

# uses the pretty okay SDK here: https://github.com/ProdPerfect/monday
from monday import MondayClient

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

In [ ]:
TASKS_BOARD_ID = "1883170887"

MONDAY_KEY = ""
environment = "dev"

In [ ]:
if not MONDAY_KEY:
    # key hasn't been passed as a papermill parameter... get it from a file?
    secrets = Box.from_yaml(filename=f"secrets-{environment}.yaml")
    MONDAY_KEY = secrets.apps.monday.API_KEY

In [ ]:
conn = MondayClient(MONDAY_KEY)

In [ ]:
# get done tasks
# tasks_df = get_items_by_board(conn, TASKS_BOARD_ID, "status", "Done")
tasks_df = get_items_by_board(conn, TASKS_BOARD_ID)

# Do not include Posted tasks
tasks_df = tasks_df.loc[
    ~tasks_df["Integration Message"].str.startswith("Posted", na=False)
]

tasks_df.rename(
    columns={
        "monday_id": "task_id",
        "monday_name": "Task Name",
        "Customer Project": "project_id",
    },
    inplace=True,
)

tasks_df = tasks_df.explode(["project_id"], ignore_index=True)
tasks_df

In [ ]:
def validate_task_record(record):
    """
    Validate checks individual records
    and we'll use those rules to create journal records later.

    Rules:
      1. Either actual hours or sessions times are used, but not both.
         If both are found, this is an error.
      2. If actual hours is used, then the number of owners dictates the number
         of journal records. E.g., actual hours = 15, with 3 owners, yields
         three journal entries at 5 each (actual hours / owner count).
         If no owners are found, this is an error.
      3. If no time fields, either actual or sessions, this is a problem.

      If session times are used, then a journal entry is created for each
         session.
    """

    if isinstance(record["Actual Time__additional_value"], list):
        sessions_list = record["Actual Time__additional_value"]
    else:
        sessions_list = []

    if isinstance(record["Owner"], list):
        owners_list = record["Owner"]
    else:
        owners_list = []

    actual_hours = record["Actual Hours"]
    len_sessions_list = len(sessions_list)
    len_owners_list = len(owners_list)
    title = record["Task Name"]
    project_id = record["project_id"]

    logger.debug(
        "actual_hours:%s, len(session_list):%s, len(owners_list):%s",
        actual_hours,
        len_sessions_list,
        len_owners_list,
    )

    # project is required
    if np.isnan(project_id) or project_id == "" or not project_id:
        record["integration_state"] = "STOP"
        record["integration_state_rule"] = "project_is_required"
        logger.warning("%s: %s", record["integration_state_rule"], title)

    # rule 1
    elif not np.isnan(actual_hours) and len_sessions_list > 0:
        record["integration_state"] = "STOP"
        record["integration_state_rule"] = "actual_hours_and_sessions"
        logger.warning("%s: %s", record["integration_state_rule"], title)

    # rule 2 - using actual hours requires at least one owner
    elif not np.isnan(actual_hours) and len_owners_list == 0:
        record["integration_state"] = "STOP"
        record["integration_state_rule"] = "actual_hours_and_no_owners"
        logger.warning("%s: %s", record["integration_state_rule"], title)

    # rule 3
    elif np.isnan(actual_hours) and len_sessions_list == 0:
        record["integration_state"] = "STOP"
        record["integration_state_rule"] = "no_actual_hours_and_no_sessions"
        logger.warning("%s: %s", record["integration_state_rule"], title)

    else:
        record["integration_state"] = "Ready"
        record["integration_state_rule"] = "Ready"

    return record

In [ ]:
# validate each record
records = tasks_df.reset_index().to_dict("records")

vald_recs = []

for record in records:
    # validate the records
    vald_rec = validate_task_record(record)
    if vald_rec:
        vald_recs.append(vald_rec)

df = pd.DataFrame(vald_recs).set_index("index")
df.head()

Use prefect and mapping to update the task/item integration status in MDC.

In [ ]:
@task(max_retries=3, retry_delay=timedelta(seconds=15))
def update_task_integration_status(monday_conn, record):
    logger = prefect.context.get("logger")
    logger.debug(f"Updating Monday.com record for {record['Task Name']}")
    monday_conn.items.change_item_value(
        TASKS_BOARD_ID,
        record["task_id"],
        "text01",
        f"{record['integration_state_rule']} - {datetime.now()}",
    )

In [ ]:
with Flow("update monday.com tasks") as flow:

    monday_conn = Parameter("monday_conn")
    validated_tasks = Parameter("validated_tasks")

    # send updates back to Monday.com... this is all one-way so no reduce required
    update_task_integration_status.map(unmapped(monday_conn), validated_tasks)

In [ ]:
params = {"monday_conn": conn, "validated_tasks": vald_recs}
state = flow.run(parameters=params, executor=LocalDaskExecutor())